In [1]:
#importing some useful packages
import pickle
import cv2
import math
import time
import h5py
import json
import os
import csv
import random
import numpy as np
import pandas as pd
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import theano
%matplotlib inline

Using TensorFlow backend.


In [12]:
df = pd.read_csv('driving_log_Arun.csv')
center = df[[0]]
center = np.squeeze(center.values)
left = df[[1]]
left = np.squeeze(left.values)
right = df[[2]]
right = np.squeeze(right.values)
steering = df[[3]]
steering = np.squeeze(steering.values)
correction = 0.25
steering_left = steering + correction
steering_right = steering - correction
print(type(center))

images = np.concatenate((np.concatenate((center, left)), right))
steer = np.concatenate((np.concatenate((steering, steering_left)), steering_right))
print(images.shape)
print (steer.shape)
batch_size = 10

def remove_spaces(path):
    len_path = len(path)
    for i in range(len_path):
        if (path[i][0] == ' '):
            path[i] = path[i][1:]
    return path

images = remove_spaces(images)

#print(images[])

<class 'numpy.ndarray'>
(24108,)
(24108,)


In [13]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    horizontal_flip=True)



In [15]:
def data_generator(Images,steering,training=0,batch_size=10,threshold=2):
    ctr=0

    while( (training and ctr<len(steering) ) or (not training) ) :
    
        if ((not training) and (ctr>=len(steering))):
            ctr=0

        data,labels=preprocess(Images[ctr:ctr+batch_size],steering[ctr:ctr+batch_size],threshold)    
        ctr +=batch_size

        yield data,labels
        
def crop(img):
    new_size_col=120
    new_size_row=60
    #img=img[60:145,:,:]
    img=cv2.resize(img,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)
    return img
        
def preprocess(Images,steering,threshold):
    row,col,ch=(mpimg.imread(Images[0])).shape

    img = []
    steer = []
        
    for i,path in enumerate(Images):

        I=mpimg.imread(path)

        #if(np.random.randint(2)):
        #    I=augment_brightness_camera_images(I)

            
        img.append(crop(I))
        steer.append(steering[i])

        #if steering[i]<-threshold or steering[i]>threshold :
        #    I=np.fliplr(I)
        #    img.append(crop(I))
        #    steer.append(-steering[i])

    return np.asarray(img),np.asarray(steer)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(images, steer, test_size = 0.2, random_state = 0)
#train_data = data_generator(X_train, y_train)
#batch_x, batch_y = next(train_data)
#print(batch_y)
#batch_x, batch_y = next(train_data)
#print(batch_y)

X = np.zeros(shape = (len(X_train), 60, 120, 3))
for i in range(len(X_train)):
    img = mpimg.imread(X_train[i])
    X[i] = crop(img)
Xtest = np.zeros(shape = (len(X_test), 60, 120, 3))
for i in range(len(X_test)):
    img = mpimg.imread(X_test[i])
    Xtest[i] = crop(img)


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
model.add(Convolution2D(128, 3, 3, border_mode = 'valid', input_shape = (60, 120, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
#model.add(Activation('softmax'))

In [22]:
model.summary()

'''
model.compile(optimizer='rmsprop',
          loss='binary_crossentropy',
          metrics=['accuracy'])
'''

model.compile(loss='mse',
              optimizer=Adam(lr = 0.001),
              metrics=['mean_squared_error'])
"""
history = model.fit(data_generator(X_train,y_train,training=0,batch_size=128),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=128),
                    nb_val_samples=len(y_test), nb_epoch=10, verbose=1)

model.fit_generator(data_generator(X_train,y_train,training=0,batch_size=64),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=64),
                    nb_val_samples=len(y_test),                    
                    nb_epoch=15,
                    verbose=2)
"""
model.fit_generator(datagen.flow(X, y_train, batch_size=128),
                    samples_per_epoch=len(X_train), validation_data = datagen.flow(Xtest, y_test, batch_size=128),
                    nb_val_samples = len(Xtest), nb_epoch = 10, verbose = 2)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 58, 118, 128)  3584        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 58, 118, 128)  0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 29, 59, 128)   0           activation_5[0][0]               
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 27, 57, 32)    36896       maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [23]:
model.save('model.h5')

In [2]:
#reading in an image
log = pd.read_csv('driving_log.csv')
X = []
print(len(log))
for n in range(100):
    img = mpimg.imread(log.ix[n][0][27:])
    left = mpimg.imread(log.ix[n][1][28:])
    right = mpimg.imread(log.ix[n][2][28:])
    new = np.concatenate((np.concatenate((left,img), axis = 2), right), axis = 2)
    new = new/255 - 0.5
    X.append(new)
X = np.asarray(X)
X_val = X
print(X.shape)
Y = np.asarray(log.ix[:99,3])
Y_val = Y
print(Y.shape)

1861
(100, 160, 320, 9)
(100,)


In [28]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode = 'valid', input_shape = (160, 320, 9)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('softmax'))

In [29]:
model.summary()

model.compile(optimizer='rmsprop',
          loss='binary_crossentropy',
          metrics=['accuracy'])
history = model.fit(X, Y, 
                    batch_size=32, nb_epoch=10, verbose=1, 
                    validation_data=(X_val,Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_10 (Convolution2D) (None, 158, 318, 32)  2624        convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 158, 318, 32)  0           convolution2d_10[0][0]           
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 79, 159, 32)   0           activation_13[0][0]              
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 77, 157, 64)   18496       maxpooling2d_7[0][0]             
___________________________________________________________________________________________

KeyboardInterrupt: 